# Read CAFE dataset directly from AWS datastore

The CAFE dataset produced by CSIRO is stored on Open Dataset on AWS. It is possible to directly read the data from the AWS store from Python. Here is an example on how to do so.

The data is organised by realm: atmos_hybrid, atmos_isobaric, ice, land, ocean, ocean_bgc, ocean_force, ocean_scalar

The list of variables in each realm is given in the [CAFE documentation](https://data.csiro.au/dap/ws/v2/collections/49803/support/4029)

The name of the AWS filesystem is S3. Some Python packages have been developed to access this filesystem. Here we are going to use s3fs.

In [1]:
import s3fs
import xarray as xr
import climtas.nci
import kerchunk.hdf
import kerchunk.combine
import dask
import ujson
import fsspec
from glob import glob
from tqdm import tqdm

## Start with an example

First, you will need to open an anonymous connection to the filesystem. The `anon` option stands for anonymous.

In [2]:
fs = s3fs.S3FileSystem(anon=True)

Let's find the path to one of the CAFE files using [the AWS Explorer](https://cafe60-reanalysis-dataset-aws-open-data.s3.amazonaws.com/index.html) and see how to open it. You can see the path in the top banner of the Explorer.

Xarray can open the files for you but you shouldn't give the file path as usual but a file object as is returned by the `open()` operation.

In [3]:
file_path = "cafe60-reanalysis-dataset-aws-open-data/atmos_isobaric/temp.atmos_isobaric.daily.CAFE60.19600101-19600131.nc"
file_obj = fs.open(file_path)   # We use fs.open() as the file is on S3
ds = xr.open_dataset(file_obj)
ds

<xarray.Dataset>
Dimensions:      (ensemble: 96, lat: 90, level: 21, lon: 144, time: 31, nv: 2)
Coordinates:
  * ensemble     (ensemble) int32 1 2 3 4 5 6 7 8 9 ... 89 90 91 92 93 94 95 96
  * lat          (lat) float64 -89.49 -87.98 -85.96 -83.93 ... 85.96 87.98 89.49
  * level        (level) float32 1.0 5.0 10.0 20.0 ... 700.0 850.0 925.0 1e+03
  * lon          (lon) float64 1.25 3.75 6.25 8.75 ... 351.2 353.8 356.2 358.8
  * time         (time) object 1960-01-01 12:00:00 ... 1960-01-31 12:00:00
Dimensions without coordinates: nv
Data variables:
    temp         (time, ensemble, level, lat, lon) float32 ...
    time_bounds  (time, nv) timedelta64[ns] ...
Attributes: (12/27)
    title:                      AccessOcean-AM2
    grid_type:                  regular
    grid_tile:                  N/A
    metadata_description:       \n\tEach of the metadata keys added via the C...
    institution:                CSIRO CAFE
    further_info_url:           https://research.csiro.au/dfp/
    ...                         ...
    run_variant_name:           data assimilation
    history_of_appended_files:  Tue May 28 12:31:55 2019: Appended file /OSM/...
    NCO:                        netCDF Operators version 4.7.8 (Homepage = ht...
    comment:                    pressure level interpolator, version 3.0, pre...
    ens_member_number:          1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,...
    filename:                   temp.atmos_isobaric.daily.CAFE60.19600101-196...

You can then use the data as usual but the operation will be longer than usual:

In [4]:
ds["temp"].mean(dim="time")

<xarray.DataArray 'temp' (ensemble: 96, level: 21, lat: 90, lon: 144)>
array([[[[260.67108, 260.67444, 260.67786, ..., 260.6621 , 260.66476,
          260.6677 ],
         [260.72556, 260.72397, 260.72308, ..., 260.73322, 260.73138,
          260.7293 ],
         [260.50287, 260.50003, 260.50076, ..., 260.52402, 260.51575,
          260.50876],
         ...,
         [215.56755, 215.54845, 215.53064, ..., 215.63278, 215.60954,
          215.58792],
         [215.83043, 215.8165 , 215.80365, ..., 215.87881, 215.8616 ,
          215.84552],
         [216.449  , 216.4476 , 216.44633, ..., 216.4532 , 216.45187,
          216.45032]],

        [[240.55174, 240.55838, 240.56522, ..., 240.53372, 240.53911,
          240.54498],
         [240.6599 , 240.65944, 240.6593 , ..., 240.66197, 240.66132,
          240.66058],
         [240.42894, 240.4416 , 240.45589, ..., 240.397  , 240.4067 ,
          240.41745],
...
         [253.26424, 253.28755, 253.32199, ..., 253.2713 , 253.2629 ,
          253.25793],
         [252.70592, 252.64792, 252.59059, ..., 252.87955, 252.81825,
          252.76454],
         [252.51163, 252.51167, 252.51173, ..., 252.51149, 252.51146,
          252.51158]],

        [[260.2997 , 260.32236, 260.3459 , ..., 260.23724, 260.2565 ,
          260.27704],
         [261.9009 , 261.89325, 261.8795 , ..., 261.9365 , 261.91452,
          261.90445],
         [262.28613, 262.12006, 261.96677, ..., 262.74078, 262.5768 ,
          262.43826],
         ...,
         [253.76541, 253.74757, 253.73332, ..., 253.90256, 253.84274,
          253.79501],
         [253.86533, 253.8579 , 253.85526, ..., 253.91324, 253.89287,
          253.8761 ],
         [253.7368 , 253.73196, 253.72743, ..., 253.75186, 253.74701,
          253.74153]]]], dtype=float32)
Coordinates:
  * ensemble  (ensemble) int32 1 2 3 4 5 6 7 8 9 ... 88 89 90 91 92 93 94 95 96
  * lat       (lat) float64 -89.49 -87.98 -85.96 -83.93 ... 85.96 87.98 89.49
  * level     (level) float32 1.0 5.0 10.0 20.0 30.0 ... 700.0 850.0 925.0 1e+03
  * lon       (lon) float64 1.25 3.75 6.25 8.75 ... 351.2 353.8 356.2 358.8

## Generalisation
From the path to the CAFE file used previously, we see that all the files are following this naming pattern:

`"cafe60-reanalysis-dataset-aws-open-data/<realm>/<varname>.<realm>.<temporal resolution>.CAFE60.<starttime>-<endtime>.nc"`

With this information, we can write a function that will return the paths to all the files for a given variable, realm and temporal resolution.

In [5]:
def find_cafe_files(realm, varname, time_res):
    """ Return a list of all the files for a given variable and a temporal resolution in a realm
    realm: str, one of the realms for the CAFE variables
    varname: str, name of one of the CAFE variables
    time_res: 'daily'|'month', temporal resolution for the variable"""
    
    root = "cafe60-reanalysis-dataset-aws-open-data"
    path = f"{root}/{realm}"
    
    my_list = fs.glob(f"{path}/{varname}.{realm}*")
    
    # Stop and return an error message if no files found.
    assert len(my_list)!=0, "There is no such variable in that realm or for that temporal resolution. Please check the CAFE documentation."
    return my_list    

In [6]:
temp_files = find_cafe_files("atmos_isobaric","temp","daily")

Let's try to read in 100 and 250 files and see how long it is. This will allow us to assess if this simple approach can work in this case

In [7]:
%%time
# Open all the files and then read them with open_mfdataset
temp_files_ob = [ fs.open(tt) for tt in temp_files ]
ds = xr.open_mfdataset(temp_files_ob[0:100],
                       combine='nested',
                       concat_dim='time', 
                       join='override',
                       coords='minimal',
                       compat='override',
                       chunks={"time":1}, 
                      )

CPU times: user 56.7 s, sys: 4.74 s, total: 1min 1s
Wall time: 2min 22s


In [ ]:
%%time
# Open all the files and then read them with open_mfdataset
temp_files_ob = [ fs.open(tt) for tt in temp_files ]
ds = xr.open_mfdataset(temp_files_ob[0:250],
                       combine='nested',
                       concat_dim='time', 
                       join='override',
                       coords='minimal',
                       compat='override',
                       chunks={"time":1}, 
                      )


The time to read 250 files is actually more than 2.5 times the time it takes to read 100 files. This tells us we can't use this simple way to read in the data in the whole timeseries of 726 files.

## Let's try to optimise this
Cloud is much better at reading a format called zarr instead of netcdf. Luckily, it's possible to fake a zarr file from a netcdf file. Let's see if it speeds things up. We can use what was done here: https://gist.github.com/rsignell-usgs/ef435a53ac530a2843ce7e1d59f96e22

For this, we need to save some data about the chunking in the netcdf files in JSON files. The `kerchunk` package was created especially for this purpose. Those JSON files only need to be created once and can be shared between users. And it's much faster to use Dask for parallelisation at this point.

In [9]:
client = climtas.nci.Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /node/ood-vn17/6784/proxy/8787/status,
Dashboard: /node/ood-vn17/6784/proxy/8787/status,Workers: 16
Total threads: 16,Total memory: 44.92 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34737,Workers: 16
Dashboard: /node/ood-vn17/6784/proxy/8787/status,Total threads: 16
Started: Just now,Total memory: 44.92 GiB
Comm: tcp://127.0.0.1:46287,Total threads: 1
Dashboard: /node/ood-vn17/6784/proxy/33447/status,Memory: 2.81 GiB
Nanny: tcp://127.0.0.1:46723,


In [ ]:
so = dict(mode='rb', anon=True, default_fill_cache=False, default_cache_type='first')

def gen_json(u, json_dir):
    with fs.open(u, **so) as infile:
        h5chunks = kerchunk.hdf.SingleHdf5ToZarr(infile, u, inline_threshold=300)
        p = u.split('/')
        fname = p[4]
        outf = f'{json_dir}/{fname}.json'
        with open(outf, 'wb') as f:
            f.write(ujson.dumps(h5chunks.translate()).encode());

In [ ]:
# Let's define the URLs and a path to save the JSON files
urls = ["s3://" + f for f in temp_files]
json_path = f"/g/data/w35/ccc561/CAFE60/json/"

In [ ]:
%%time
dask.compute(*[dask.delayed(gen_json)(u, json_path) for u in urls], retries=10);

Now that we have encoded the data we need in JSON files, we can use those to read in the entire dataset. We first need to create a mapping between the JSON files and the data on S3. Then we use `xarray.open_mfdataset()` on this mapping. Note, we then need to specify the `zarr` engine for reading the data.

In [11]:
# Get a list of the json files we need
json_path = f"/g/data/w35/ccc561/CAFE60/json/"
json_list = sorted(glob(f"{json_path}/temp*.json"))

In [12]:
# Get the mapping
m_list = []
for js in tqdm(json_list):
    with open(js) as f:
        m_list.append(fsspec.get_mapper("reference://",
                                fo=ujson.load(f),
                                remote_protocol="s3",
                                remote_options={"anon":True}))

100%|██████████| 726/726 [00:09<00:00, 78.28it/s] 


In [15]:
%%time
# Open the data with the zarr engine from the mapping objects m_list
ds1 = xr.open_mfdataset(m_list,
                        combine="nested",
                        concat_dim="time",
                        engine="zarr",
                        coords="minimal",
                        data_vars="minimal",
                        compat="override",
                        parallel=True
                       )

CPU times: user 20.7 s, sys: 3.45 s, total: 24.1 s
Wall time: 26.5 s


We get a fair amount of warnings but all the files for the timeseries are open in less than a minute.

In [16]:
ds1

<xarray.Dataset>
Dimensions:      (latb: 133, lonb: 145, nv: 2, ensemble: 96, lat: 90, level: 21, lon: 144, time: 22097)
Coordinates:
  * latb         (latb) float64 -90.0 -88.99 -86.97 -84.94 ... 86.97 88.99 90.0
  * lonb         (lonb) float64 0.0 2.5 5.0 7.5 10.0 ... 352.5 355.0 357.5 360.0
  * nv           (nv) float64 1.0 2.0
  * ensemble     (ensemble) float64 1.0 2.0 3.0 4.0 5.0 ... 93.0 94.0 95.0 96.0
  * lat          (lat) float64 -89.49 -87.98 -85.96 -83.93 ... 85.96 87.98 89.49
  * level        (level) float32 1.0 5.0 10.0 20.0 ... 700.0 850.0 925.0 1e+03
  * lon          (lon) float64 1.25 3.75 6.25 8.75 ... 351.2 353.8 356.2 358.8
  * time         (time) object 1960-01-01 12:00:00 ... 2020-11-30 12:00:00
Data variables:
    temp         (time, ensemble, level, lat, lon) float32 dask.array<chunksize=(1, 96, 7, 31, 49), meta=np.ndarray>
    time_bounds  (time, nv) timedelta64[ns] dask.array<chunksize=(1, 2), meta=np.ndarray>
Attributes: (12/27)
    NCO:                        netCDF Operators version 4.7.8 (Homepage = ht...
    calendar:                    julian
    cm-enkf_source:             commit 0141830a243704acbf6d8c8c843b161a39ec39fa
    comment:                    pressure level interpolator, version 3.0, pre...
    contact_name:               Decadal Activity 1 - Data Assimilation
    control_name:               c5
    ...                         ...
    model_source:               commit 2abb29f4384e68777721fa657850587f28efd85a
    nominal_resoltuion:         Atmosphere delta lat = 2.02degrees ; Atmosphe...
    perturbation_name:          not applicable
    references:                 OKane, T.J., Sandery, P.A., Monselesan, D.P.,...
    run_variant_name:           data assimilation
    title:                      AccessOcean-AM2

Because Python will need to go and read the data on the cloud for each computation, it is slower than with local data. Below is an example of the time it would take to create an ensemble mean of half the timeseries, at one level roughly over Australia.

In [17]:
temp1 = ds1["temp"].isel(level=0,time=slice(0,10000)).sel(lon=slice(120,150), lat=slice(-45,-5))
temp1_mean = temp1.mean(dim="ensemble")
temp1_mean

<xarray.DataArray 'temp' (time: 10000, lat: 20, lon: 12)>
dask.array<mean_agg-aggregate, shape=(10000, 20, 12), dtype=float32, chunksize=(1, 12, 11), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 -43.48 -41.46 -39.44 -37.42 ... -9.101 -7.079 -5.056
    level    float32 1.0
  * lon      (lon) float64 121.2 123.8 126.2 128.8 ... 141.2 143.8 146.2 148.8
  * time     (time) object 1960-01-01 12:00:00 ... 1987-05-18 12:00:00

In [18]:
%%time
temp1_mean.load()

CPU times: user 8min 53s, sys: 1min 19s, total: 10min 13s
Wall time: 12min 38s


<xarray.DataArray 'temp' (time: 10000, lat: 20, lon: 12)>
array([[[235.98643, 236.0106 , 236.02881, ..., 236.09534, 236.07268,
         236.04564],
        [237.92964, 237.92836, 237.92194, ..., 237.76111, 237.70045,
         237.63062],
        [239.72624, 239.70036, 239.67493, ..., 239.3267 , 239.25848,
         239.17676],
        ...,
        [258.3558 , 258.3583 , 258.33548, ..., 257.89304, 257.83365,
         257.88174],
        [258.9962 , 259.01123, 259.00974, ..., 258.38364, 258.42105,
         258.45718],
        [259.57407, 259.62177, 259.62582, ..., 258.81683, 258.80463,
         258.77777]],

       [[240.83653, 240.89354, 240.95435, ..., 241.2726 , 241.29076,
         241.30333],
        [242.51823, 242.5636 , 242.60957, ..., 242.74242, 242.72713,
         242.70874],
        [244.10077, 244.12251, 244.14238, ..., 244.10237, 244.08421,
         244.04791],
...
        [259.68243, 259.63416, 259.61905, ..., 259.52643, 259.5286 ,
         259.53375],
        [260.20752, 260.18234, 260.2126 , ..., 260.10892, 260.13065,
         260.2048 ],
        [260.702  , 260.718  , 260.7333 , ..., 260.57236, 260.62637,
         260.73703]],

       [[234.87727, 234.89783, 234.90929, ..., 234.8871 , 234.8867 ,
         234.85811],
        [236.40094, 236.39925, 236.39099, ..., 236.32133, 236.31029,
         236.2804 ],
        [237.8614 , 237.83423, 237.8133 , ..., 237.77417, 237.80426,
         237.74947],
        ...,
        [259.54483, 259.52542, 259.5184 , ..., 259.49985, 259.4794 ,
         259.4852 ],
        [260.0524 , 260.05325, 260.06937, ..., 260.08627, 260.08524,
         260.17264],
        [260.4982 , 260.53256, 260.55963, ..., 260.5532 , 260.60464,
         260.73898]]], dtype=float32)
Coordinates:
  * lat      (lat) float64 -43.48 -41.46 -39.44 -37.42 ... -9.101 -7.079 -5.056
    level    float32 1.0
  * lon      (lon) float64 121.2 123.8 126.2 128.8 ... 141.2 143.8 146.2 148.8
  * time     (time) object 1960-01-01 12:00:00 ... 1987-05-18 12:00:00

## Conclusion
We can store the JSON files for the required variables centrally on ua8. Then users can access the data in this way:

In [ ]:
from glob import glob
import xarray as xr
import ujson

# Get a list of the json files we need
json_path = f"/g/data/w35/ccc561/CAFE60/json/"  # Update the path to the central location
json_list = sorted(glob(f"{json_path}/temp*.json"))  # Update the path to the variable you need.

# Get the mapping
m_list = []
for js in json_list:
    with open(js) as f:
        m_list.append(fsspec.get_mapper("reference://",
                                fo=ujson.load(f),
                                remote_protocol="s3",
                                remote_options={"anon":True}))
ds1 = xr.open_mfdataset(m_list,
                        combine="nested",
                        concat_dim="time",
                        engine="zarr",
                        coords="minimal",
                        data_vars="minimal",
                        compat="override",
                        parallel=True
                       )